In [1]:
 %matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os

from sklearn import tree

In [2]:
data = pd.read_csv('./Resources/WA_Fn-UseC_-Marketing-Customer-Value-Analysis.csv')
data.head()

,Customer,State,Customer Lifetime Value,Response,Coverage,Education,Effective To Date,EmploymentStatus,Gender,Income,...,Months Since Policy Inception,Number of Open Complaints,Number of Policies,Policy Type,Policy,Renew Offer Type,Sales Channel,Total Claim Amount,Vehicle Class,Vehicle Size
0,BU79786,Washington,2763.519279,No,Basic,Bachelor,2/24/11,Employed,F,56274,...,5,0,1,Corporate Auto,Corporate L3,Offer1,Agent,384.811147,Two-Door Car,Medsize
1,QZ44356,Arizona,6979.535903,No,Extended,Bachelor,1/31/11,Unemployed,F,0,...,42,0,8,Personal Auto,Personal L3,Offer3,Agent,1131.464935,Four-Door Car,Medsize
2,AI49188,Nevada,12887.431650,No,Premium,Bachelor,2/19/11,Employed,F,48767,...,38,0,2,Personal Auto,Personal L3,Offer1,Agent,566.472247,Two-Door Car,Medsize
3,WW63253,California,7645.861827,No,Basic,Bachelor,1/20/11,Unemployed,M,0,...,65,0,7,Corporate Auto,Corporate L2,Offer1,Call Center,529.881344,SUV,Medsize
4,HB64268,Washington,2813.692575,No,Basic,Bachelor,2/3/11,Employed,M,43836,...,44,0,1,Personal Auto,Personal L1,Offer1,Agent,138.130879,Four-Door Car,Medsize


In [3]:
new_data = data.drop(columns=['Effective To Date', 'Customer'])
new_data

,State,Customer Lifetime Value,Response,Coverage,Education,EmploymentStatus,Gender,Income,Location Code,Marital Status,...,Months Since Policy Inception,Number of Open Complaints,Number of Policies,Policy Type,Policy,Renew Offer Type,Sales Channel,Total Claim Amount,Vehicle Class,Vehicle Size
0,Washington,2763.519279,No,Basic,Bachelor,Employed,F,56274,Suburban,Married,...,5,0,1,Corporate Auto,Corporate L3,Offer1,Agent,384.811147,Two-Door Car,Medsize
1,Arizona,6979.535903,No,Extended,Bachelor,Unemployed,F,0,Suburban,Single,...,42,0,8,Personal Auto,Personal L3,Offer3,Agent,1131.464935,Four-Door Car,Medsize
2,Nevada,12887.431650,No,Premium,Bachelor,Employed,F,48767,Suburban,Married,...,38,0,2,Personal Auto,Personal L3,Offer1,Agent,566.472247,Two-Door Car,Medsize
3,California,7645.861827,No,Basic,Bachelor,Unemployed,M,0,Suburban,Married,...,65,0,7,Corporate Auto,Corporate L2,Offer1,Call Center,529.881344,SUV,Medsize
4,Washington,2813.692575,No,Basic,Bachelor,Employed,M,43836,Rural,Single,...,44,0,1,Personal Auto,Personal L1,Offer1,Agent,138.130879,Four-Door Car,Medsize
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9129,California,23405.987980,No,Basic,Bachelor,Employed,M,71941,Urban,Married,...,89,0,2,Personal Auto,Personal L1,Offer2,Web,198.234764,Four-Door Car,Medsize
9130,California,3096.511217,Yes,Extended,College,Employed,F,21604,Suburban,Divorced,...,28,0,1,Corporate Auto,Corporate L3,Offer1,Branch,379.200000,Four-Door Car,Medsize
9131,California,8163.890428,No,Extended,Bachelor,Unemployed,M,0,Suburban,Single,...,37,3,2,Corporate Auto,Corporate L2,Offer1,Branch,790.784983,Four-Door Car,Medsize
9132,California,7524.442436,No,Extended,College,Employed,M,21941,Suburban,Married,...,3,0,3,Personal Auto,Personal L2,Offer3,Branch,691.200000,Four-Door Car,Large


In [4]:
# Add a column of 'Quartile LTV'. This column will determine the quartile where each 
# customer falls in thems of Customer Life Time Value.

# First, find the quartile threshold values

ltv = new_data["Customer Lifetime Value"]  

print("Q3 quantile of arr : ", np.quantile(ltv, .25)) 
print("Q2 quantile of arr : ", np.quantile(ltv, .50))  
print("Q1 quantile of arr : ", np.quantile(ltv, .75)) 
 


Q3 quantile of arr :  3994.25179425
Q2 quantile of arr :  5780.182197
Q1 quantile of arr :  8962.16704125


In [5]:
# Create new column with conditions based on quartlie thresholds
new_data.loc[(new_data['Customer Lifetime Value'] >0) & (new_data['Customer Lifetime Value']<= 3994.26), 'Quartile LTV'] = 4 
new_data.loc[(new_data['Customer Lifetime Value'] > 3994.26)& (new_data['Customer Lifetime Value'] <= 5780.18), 'Quartile LTV'] = 3
new_data.loc[(new_data['Customer Lifetime Value'] > 5780.19) & (new_data['Customer Lifetime Value']<= 8962.16), 'Quartile LTV'] = 2
new_data.loc[(new_data['Customer Lifetime Value'] > 8962.17), 'Quartile LTV'] = 1
new_data.head(50)

,State,Customer Lifetime Value,Response,Coverage,Education,EmploymentStatus,Gender,Income,Location Code,Marital Status,...,Number of Open Complaints,Number of Policies,Policy Type,Policy,Renew Offer Type,Sales Channel,Total Claim Amount,Vehicle Class,Vehicle Size,Quartile LTV
0,Washington,2763.519279,No,Basic,Bachelor,Employed,F,56274,Suburban,Married,...,0,1,Corporate Auto,Corporate L3,Offer1,Agent,384.811147,Two-Door Car,Medsize,4.0
1,Arizona,6979.535903,No,Extended,Bachelor,Unemployed,F,0,Suburban,Single,...,0,8,Personal Auto,Personal L3,Offer3,Agent,1131.464935,Four-Door Car,Medsize,2.0
2,Nevada,12887.431650,No,Premium,Bachelor,Employed,F,48767,Suburban,Married,...,0,2,Personal Auto,Personal L3,Offer1,Agent,566.472247,Two-Door Car,Medsize,1.0
3,California,7645.861827,No,Basic,Bachelor,Unemployed,M,0,Suburban,Married,...,0,7,Corporate Auto,Corporate L2,Offer1,Call Center,529.881344,SUV,Medsize,2.0
4,Washington,2813.692575,No,Basic,Bachelor,Employed,M,43836,Rural,Single,...,0,1,Personal Auto,Personal L1,Offer1,Agent,138.130879,Four-Door Car,Medsize,4.0
5,Oregon,8256.297800,Yes,Basic,Bachelor,Employed,F,62902,Rural,Married,...,0,2,Personal Auto,Personal L3,Offer2,Web,159.383042,Two-Door Car,Medsize,2.0
6,Oregon,5380.898636,Yes,Basic,College,Employed,F,55350,Suburban,Married,...,0,9,Corporate Auto,Corporate L3,Offer1,Agent,321.600000,Four-Door Car,Medsize,3.0
7,Arizona,7216.100311,No,Premium,Master,Unemployed,M,0,Urban,Single,...,0,4,Corporate Auto,Corporate L3,Offer1,Agent,363.029680,Four-Door Car,Medsize,2.0
8,Oregon,24127.504020,Yes,Basic,Bachelor,Medical Leave,M,14072,Suburban,Divorced,...,0,2,Corporate Auto,Corporate L3,Offer1,Agent,511.200000,Four-Door Car,Medsize,1.0
9,Oregon,7388.178085,No,Extended,College,Employed,F,28812,Urban,Married,...,0,8,Special Auto,Special L2,Offer2,Branch,425.527834,Four-Door Car,Medsize,2.0


In [39]:
new_data_quartileLTV = new_data.drop(columns=['Customer Lifetime Value'])
new_data_quartileLTV.head()

,State,Response,Coverage,Education,EmploymentStatus,Gender,Income,Location Code,Marital Status,Monthly Premium Auto,...,Number of Open Complaints,Number of Policies,Policy Type,Policy,Renew Offer Type,Sales Channel,Total Claim Amount,Vehicle Class,Vehicle Size,Quartile LTV
0,Washington,No,Basic,Bachelor,Employed,F,56274,Suburban,Married,69,...,0,1,Corporate Auto,Corporate L3,Offer1,Agent,384.811147,Two-Door Car,Medsize,4.0
1,Arizona,No,Extended,Bachelor,Unemployed,F,0,Suburban,Single,94,...,0,8,Personal Auto,Personal L3,Offer3,Agent,1131.464935,Four-Door Car,Medsize,2.0
2,Nevada,No,Premium,Bachelor,Employed,F,48767,Suburban,Married,108,...,0,2,Personal Auto,Personal L3,Offer1,Agent,566.472247,Two-Door Car,Medsize,1.0
3,California,No,Basic,Bachelor,Unemployed,M,0,Suburban,Married,106,...,0,7,Corporate Auto,Corporate L2,Offer1,Call Center,529.881344,SUV,Medsize,2.0
4,Washington,No,Basic,Bachelor,Employed,M,43836,Rural,Single,73,...,0,1,Personal Auto,Personal L1,Offer1,Agent,138.130879,Four-Door Car,Medsize,4.0


In [25]:
# Inputs to neural networks are vectors. Each entry in the vector corresponds to a feature, which the net 
# uses to make predictions.Crucially, these vectors contain can contain only numerical data. They cannot 
# contain string data.

# Convert each categorical variable to a numerical value. We do this via the following steps:
# Label Encoding. First, we convert the possible classes to integer labels for each categorical variable.  
# One-Hot Encoding. Then, we set each row's value to an array. This array will have a 1 in whichever slot 
# corresponds to the integer label.

test = new_data_quartileLTV['State']


test



0       Washington
1          Arizona
2           Nevada
3       California
4       Washington
           ...    
9129    California
9130    California
9131    California
9132    California
9133    California
Name: State, Length: 9134, dtype: object

In [26]:
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
label_encoder.fit(test)

encoded_test = label_encoder.transform(test)

In [27]:
for label, original_class in zip(encoded_test, test):
    print('Original Class: ' + str(original_class))
    print('Encoded Label: ' + str(label))
    print('-' * 12)

Original Class: Washington
Encoded Label: 4
------------
Original Class: Arizona
Encoded Label: 0
------------
Original Class: Nevada
Encoded Label: 2
------------
Original Class: California
Encoded Label: 1
------------
Original Class: Washington
Encoded Label: 4
------------
Original Class: Oregon
Encoded Label: 3
------------
Original Class: Oregon
Encoded Label: 3
------------
Original Class: Arizona
Encoded Label: 0
------------
Original Class: Oregon
Encoded Label: 3
------------
Original Class: Oregon
Encoded Label: 3
------------
Original Class: California
Encoded Label: 1
------------
Original Class: California
Encoded Label: 1
------------
Original Class: California
Encoded Label: 1
------------
Original Class: Arizona
Encoded Label: 0
------------
Original Class: California
Encoded Label: 1
------------
Original Class: Oregon
Encoded Label: 3
------------
Original Class: Nevada
Encoded Label: 2
------------
Original Class: California
Encoded Label: 1
------------
Original Cl

Encoded Label: 0
------------
Original Class: California
Encoded Label: 1
------------
Original Class: Oregon
Encoded Label: 3
------------
Original Class: Washington
Encoded Label: 4
------------
Original Class: California
Encoded Label: 1
------------
Original Class: California
Encoded Label: 1
------------
Original Class: California
Encoded Label: 1
------------
Original Class: California
Encoded Label: 1
------------
Original Class: Oregon
Encoded Label: 3
------------
Original Class: Arizona
Encoded Label: 0
------------
Original Class: Arizona
Encoded Label: 0
------------
Original Class: Oregon
Encoded Label: 3
------------
Original Class: Oregon
Encoded Label: 3
------------
Original Class: California
Encoded Label: 1
------------
Original Class: Oregon
Encoded Label: 3
------------
Original Class: Oregon
Encoded Label: 3
------------
Original Class: California
Encoded Label: 1
------------
Original Class: Oregon
Encoded Label: 3
------------
Original Class: Washington
Encoded 

------------
Original Class: Arizona
Encoded Label: 0
------------
Original Class: Oregon
Encoded Label: 3
------------
Original Class: California
Encoded Label: 1
------------
Original Class: Washington
Encoded Label: 4
------------
Original Class: California
Encoded Label: 1
------------
Original Class: California
Encoded Label: 1
------------
Original Class: Oregon
Encoded Label: 3
------------
Original Class: California
Encoded Label: 1
------------
Original Class: Oregon
Encoded Label: 3
------------
Original Class: Oregon
Encoded Label: 3
------------
Original Class: Arizona
Encoded Label: 0
------------
Original Class: Nevada
Encoded Label: 2
------------
Original Class: Arizona
Encoded Label: 0
------------
Original Class: California
Encoded Label: 1
------------
Original Class: Arizona
Encoded Label: 0
------------
Original Class: California
Encoded Label: 1
------------
Original Class: California
Encoded Label: 1
------------
Original Class: Oregon
Encoded Label: 3
----------

Original Class: Arizona
Encoded Label: 0
------------
Original Class: Oregon
Encoded Label: 3
------------
Original Class: Nevada
Encoded Label: 2
------------
Original Class: California
Encoded Label: 1
------------
Original Class: California
Encoded Label: 1
------------
Original Class: California
Encoded Label: 1
------------
Original Class: Oregon
Encoded Label: 3
------------
Original Class: Oregon
Encoded Label: 3
------------
Original Class: Arizona
Encoded Label: 0
------------
Original Class: Oregon
Encoded Label: 3
------------
Original Class: Nevada
Encoded Label: 2
------------
Original Class: Arizona
Encoded Label: 0
------------
Original Class: Arizona
Encoded Label: 0
------------
Original Class: California
Encoded Label: 1
------------
Original Class: California
Encoded Label: 1
------------
Original Class: California
Encoded Label: 1
------------
Original Class: Oregon
Encoded Label: 3
------------
Original Class: Oregon
Encoded Label: 3
------------
Original Class: Or

Encoded Label: 0
------------
Original Class: Oregon
Encoded Label: 3
------------
Original Class: Oregon
Encoded Label: 3
------------
Original Class: California
Encoded Label: 1
------------
Original Class: Oregon
Encoded Label: 3
------------
Original Class: Arizona
Encoded Label: 0
------------
Original Class: California
Encoded Label: 1
------------
Original Class: Oregon
Encoded Label: 3
------------
Original Class: California
Encoded Label: 1
------------
Original Class: Arizona
Encoded Label: 0
------------
Original Class: California
Encoded Label: 1
------------
Original Class: California
Encoded Label: 1
------------
Original Class: California
Encoded Label: 1
------------
Original Class: Arizona
Encoded Label: 0
------------
Original Class: California
Encoded Label: 1
------------
Original Class: California
Encoded Label: 1
------------
Original Class: California
Encoded Label: 1
------------
Original Class: Oregon
Encoded Label: 3
------------
Original Class: Oregon
Encoded

Encoded Label: 3
------------
Original Class: Oregon
Encoded Label: 3
------------
Original Class: Nevada
Encoded Label: 2
------------
Original Class: Arizona
Encoded Label: 0
------------
Original Class: California
Encoded Label: 1
------------
Original Class: Oregon
Encoded Label: 3
------------
Original Class: Arizona
Encoded Label: 0
------------
Original Class: California
Encoded Label: 1
------------
Original Class: California
Encoded Label: 1
------------
Original Class: California
Encoded Label: 1
------------
Original Class: Washington
Encoded Label: 4
------------
Original Class: California
Encoded Label: 1
------------
Original Class: Nevada
Encoded Label: 2
------------
Original Class: California
Encoded Label: 1
------------
Original Class: Arizona
Encoded Label: 0
------------
Original Class: Arizona
Encoded Label: 0
------------
Original Class: Oregon
Encoded Label: 3
------------
Original Class: Nevada
Encoded Label: 2
------------
Original Class: California
Encoded La

Encoded Label: 3
------------
Original Class: Oregon
Encoded Label: 3
------------
Original Class: Oregon
Encoded Label: 3
------------
Original Class: California
Encoded Label: 1
------------
Original Class: Oregon
Encoded Label: 3
------------
Original Class: Nevada
Encoded Label: 2
------------
Original Class: Nevada
Encoded Label: 2
------------
Original Class: Arizona
Encoded Label: 0
------------
Original Class: California
Encoded Label: 1
------------
Original Class: California
Encoded Label: 1
------------
Original Class: California
Encoded Label: 1
------------
Original Class: Oregon
Encoded Label: 3
------------
Original Class: Arizona
Encoded Label: 0
------------
Original Class: Oregon
Encoded Label: 3
------------
Original Class: Washington
Encoded Label: 4
------------
Original Class: Nevada
Encoded Label: 2
------------
Original Class: Oregon
Encoded Label: 3
------------
Original Class: Arizona
Encoded Label: 0
------------
Original Class: Washington
Encoded Label: 4
--

Encoded Label: 1
------------
Original Class: California
Encoded Label: 1
------------
Original Class: California
Encoded Label: 1
------------
Original Class: California
Encoded Label: 1
------------
Original Class: California
Encoded Label: 1
------------
Original Class: California
Encoded Label: 1
------------
Original Class: California
Encoded Label: 1
------------
Original Class: California
Encoded Label: 1
------------
Original Class: California
Encoded Label: 1
------------
Original Class: California
Encoded Label: 1
------------
Original Class: California
Encoded Label: 1
------------
Original Class: California
Encoded Label: 1
------------
Original Class: California
Encoded Label: 1
------------
Original Class: California
Encoded Label: 1
------------
Original Class: California
Encoded Label: 1
------------
Original Class: California
Encoded Label: 1
------------
Original Class: California
Encoded Label: 1
------------
Original Class: California
Encoded Label: 1
------------
O

In [28]:
from keras.utils import to_categorical

# Step 2: One-hot encoding
one_hot_test = to_categorical(encoded_test)

one_hot_test

array([[0., 0., 0., 0., 1.],
       [1., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0.],
       ...,
       [0., 1., 0., 0., 0.],
       [0., 1., 0., 0., 0.],
       [0., 1., 0., 0., 0.]], dtype=float32)

In [42]:
new_data_quartileLTV.head(15)

,State,Response,Coverage,Education,EmploymentStatus,Gender,Income,Location Code,Marital Status,Monthly Premium Auto,...,Number of Open Complaints,Number of Policies,Policy Type,Policy,Renew Offer Type,Sales Channel,Total Claim Amount,Vehicle Class,Vehicle Size,Quartile LTV
0,0.0,No,Basic,Bachelor,Employed,F,56274,Suburban,Married,69,...,0,1,Corporate Auto,Corporate L3,Offer1,Agent,384.811147,Two-Door Car,Medsize,4.0
1,1.0,No,Extended,Bachelor,Unemployed,F,0,Suburban,Single,94,...,0,8,Personal Auto,Personal L3,Offer3,Agent,1131.464935,Four-Door Car,Medsize,2.0
2,0.0,No,Premium,Bachelor,Employed,F,48767,Suburban,Married,108,...,0,2,Personal Auto,Personal L3,Offer1,Agent,566.472247,Two-Door Car,Medsize,1.0
3,0.0,No,Basic,Bachelor,Unemployed,M,0,Suburban,Married,106,...,0,7,Corporate Auto,Corporate L2,Offer1,Call Center,529.881344,SUV,Medsize,2.0
4,0.0,No,Basic,Bachelor,Employed,M,43836,Rural,Single,73,...,0,1,Personal Auto,Personal L1,Offer1,Agent,138.130879,Four-Door Car,Medsize,4.0
5,0.0,Yes,Basic,Bachelor,Employed,F,62902,Rural,Married,69,...,0,2,Personal Auto,Personal L3,Offer2,Web,159.383042,Two-Door Car,Medsize,2.0
6,0.0,Yes,Basic,College,Employed,F,55350,Suburban,Married,67,...,0,9,Corporate Auto,Corporate L3,Offer1,Agent,321.600000,Four-Door Car,Medsize,3.0
7,1.0,No,Premium,Master,Unemployed,M,0,Urban,Single,101,...,0,4,Corporate Auto,Corporate L3,Offer1,Agent,363.029680,Four-Door Car,Medsize,2.0
8,0.0,Yes,Basic,Bachelor,Medical Leave,M,14072,Suburban,Divorced,71,...,0,2,Corporate Auto,Corporate L3,Offer1,Agent,511.200000,Four-Door Car,Medsize,1.0
9,0.0,No,Extended,College,Employed,F,28812,Urban,Married,93,...,0,8,Special Auto,Special L2,Offer2,Branch,425.527834,Four-Door Car,Medsize,2.0


In [40]:
new_data_quartileLTV['State'] = one_hot_test

In [41]:
new_data_quartileLTV.head(15)

,State,Response,Coverage,Education,EmploymentStatus,Gender,Income,Location Code,Marital Status,Monthly Premium Auto,...,Number of Open Complaints,Number of Policies,Policy Type,Policy,Renew Offer Type,Sales Channel,Total Claim Amount,Vehicle Class,Vehicle Size,Quartile LTV
0,0.0,No,Basic,Bachelor,Employed,F,56274,Suburban,Married,69,...,0,1,Corporate Auto,Corporate L3,Offer1,Agent,384.811147,Two-Door Car,Medsize,4.0
1,1.0,No,Extended,Bachelor,Unemployed,F,0,Suburban,Single,94,...,0,8,Personal Auto,Personal L3,Offer3,Agent,1131.464935,Four-Door Car,Medsize,2.0
2,0.0,No,Premium,Bachelor,Employed,F,48767,Suburban,Married,108,...,0,2,Personal Auto,Personal L3,Offer1,Agent,566.472247,Two-Door Car,Medsize,1.0
3,0.0,No,Basic,Bachelor,Unemployed,M,0,Suburban,Married,106,...,0,7,Corporate Auto,Corporate L2,Offer1,Call Center,529.881344,SUV,Medsize,2.0
4,0.0,No,Basic,Bachelor,Employed,M,43836,Rural,Single,73,...,0,1,Personal Auto,Personal L1,Offer1,Agent,138.130879,Four-Door Car,Medsize,4.0
5,0.0,Yes,Basic,Bachelor,Employed,F,62902,Rural,Married,69,...,0,2,Personal Auto,Personal L3,Offer2,Web,159.383042,Two-Door Car,Medsize,2.0
6,0.0,Yes,Basic,College,Employed,F,55350,Suburban,Married,67,...,0,9,Corporate Auto,Corporate L3,Offer1,Agent,321.600000,Four-Door Car,Medsize,3.0
7,1.0,No,Premium,Master,Unemployed,M,0,Urban,Single,101,...,0,4,Corporate Auto,Corporate L3,Offer1,Agent,363.029680,Four-Door Car,Medsize,2.0
8,0.0,Yes,Basic,Bachelor,Medical Leave,M,14072,Suburban,Divorced,71,...,0,2,Corporate Auto,Corporate L3,Offer1,Agent,511.200000,Four-Door Car,Medsize,1.0
9,0.0,No,Extended,College,Employed,F,28812,Urban,Married,93,...,0,8,Special Auto,Special L2,Offer2,Branch,425.527834,Four-Door Car,Medsize,2.0


In [ ]:
## Try with pandas Dummies
new_data_quartileLTV['State'] = new_data_quartileLTV['State'].astype('category')
new_data_quartileLTV = pd.get_dummies(new_data_quartileLTV)


In [ ]:
new_data_quartileLTV.head()

In [7]:
new_data.loc[(new_data['Customer Lifetime Value'] >0) & (new_data['Customer Lifetime Value']<= 3994.26), 'Quartile LTV'] = 4 
new_data.loc[(new_data['Customer Lifetime Value'] > 3994.26)& (new_data['Customer Lifetime Value'] <= 5780.18), 'Quartile LTV'] = 3
new_data.loc[(new_data['Customer Lifetime Value'] > 5780.19) & (new_data['Customer Lifetime Value']<= 8962.16), 'Quartile LTV'] = 2
new_data.loc[(new_data['Customer Lifetime Value'] > 8962.17), 'Quartile LTV'] = 1
new_data.head(50)

,State,Customer Lifetime Value,Response,Coverage,Education,EmploymentStatus,Gender,Income,Location Code,Marital Status,...,Number of Open Complaints,Number of Policies,Policy Type,Policy,Renew Offer Type,Sales Channel,Total Claim Amount,Vehicle Class,Vehicle Size,Quartile LTV
0,Washington,2763.519279,No,Basic,Bachelor,Employed,F,56274,Suburban,Married,...,0,1,Corporate Auto,Corporate L3,Offer1,Agent,384.811147,Two-Door Car,Medsize,4.0
1,Arizona,6979.535903,No,Extended,Bachelor,Unemployed,F,0,Suburban,Single,...,0,8,Personal Auto,Personal L3,Offer3,Agent,1131.464935,Four-Door Car,Medsize,2.0
2,Nevada,12887.431650,No,Premium,Bachelor,Employed,F,48767,Suburban,Married,...,0,2,Personal Auto,Personal L3,Offer1,Agent,566.472247,Two-Door Car,Medsize,1.0
3,California,7645.861827,No,Basic,Bachelor,Unemployed,M,0,Suburban,Married,...,0,7,Corporate Auto,Corporate L2,Offer1,Call Center,529.881344,SUV,Medsize,2.0
4,Washington,2813.692575,No,Basic,Bachelor,Employed,M,43836,Rural,Single,...,0,1,Personal Auto,Personal L1,Offer1,Agent,138.130879,Four-Door Car,Medsize,4.0
5,Oregon,8256.297800,Yes,Basic,Bachelor,Employed,F,62902,Rural,Married,...,0,2,Personal Auto,Personal L3,Offer2,Web,159.383042,Two-Door Car,Medsize,2.0
6,Oregon,5380.898636,Yes,Basic,College,Employed,F,55350,Suburban,Married,...,0,9,Corporate Auto,Corporate L3,Offer1,Agent,321.600000,Four-Door Car,Medsize,3.0
7,Arizona,7216.100311,No,Premium,Master,Unemployed,M,0,Urban,Single,...,0,4,Corporate Auto,Corporate L3,Offer1,Agent,363.029680,Four-Door Car,Medsize,2.0
8,Oregon,24127.504020,Yes,Basic,Bachelor,Medical Leave,M,14072,Suburban,Divorced,...,0,2,Corporate Auto,Corporate L3,Offer1,Agent,511.200000,Four-Door Car,Medsize,1.0
9,Oregon,7388.178085,No,Extended,College,Employed,F,28812,Urban,Married,...,0,8,Special Auto,Special L2,Offer2,Branch,425.527834,Four-Door Car,Medsize,2.0
